In [1]:
import pandas as pd
import numpy as np
import csv
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, StandardScaler
from sklearn.model_selection import train_test_split, ParameterGrid
from sklearn.metrics import accuracy_score, confusion_matrix, ConfusionMatrixDisplay
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

- `job`
- `education`
- `device`
- `outcome_old`

# import data

In [2]:
campaign_ad = pd.read_csv("MLUnige2023_subscriptions_train.csv", index_col="Id")
campaign_test = pd.read_csv("MLUnige2023_subscriptions_test.csv", index_col="Id")

# dummify marital and outcome_old

In [3]:
campaign_ad = pd.get_dummies(campaign_ad, columns=['marital', 'outcome_old'])

# train-valid-test split

In [4]:
X = campaign_ad.drop(columns='subscription')
y = campaign_ad['subscription']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=12)
X_valid, X_test, y_valid, y_test = train_test_split(X_test, y_test, test_size=0.5, random_state=46)

In [5]:
print('size of training set:', X_train.shape[0])
print('size of validation set:', X_valid.shape[0])
print('size of test set:', X_test.shape[0])

size of training set: 6266
size of validation set: 1343
size of test set: 1343


# imputation for JOB

In [6]:
X_tr_job = X_train.drop(columns=['device', 'education'])

In [7]:
X_test_job = X_tr_job[X_tr_job['job'] == 'na'].drop(columns='job')

In [8]:
X_tr_job = X_tr_job[X_tr_dev['job'] != 'na']
y_tr_job = X_tr_dev[['job']]
X_tr_dev = X_tr_dev.drop(columns='job')

NameError: name 'X_tr_dev' is not defined

In [ ]:
enc = OrdinalEncoder()
y_tr_job = enc.fit_transform(y_tr_job)

In [ ]:
rfc = RandomForestClassifier(n_estimators=100, random_state=59, n_jobs=-2)

In [ ]:
rfc.fit(X_tr_dev, y_tr_job)

C:\Users\ficop\AppData\Local\Temp\ipykernel_23012\3946294562.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc.fit(X_tr_job, y_tr_job)


RandomForestClassifier(n_jobs=-2, random_state=59)

In [ ]:
y_tr_job_pred = rfc.predict(X_tr_job)
accuracy_score(y_true=y_tr_job, y_pred=y_tr_job_pred)

1.0

In [ ]:
X_v_job = X_valid.drop(columns=['device', 'education'])
X_test_job = X_v_job[X_v_job['job'] == 'na'].drop(columns='job')
X_v_job = X_v_job[X_v_job['job'] != 'na']
y_v_job = X_v_job[['job']]
X_v_job = X_v_job.drop(columns='job')
enc = OrdinalEncoder()
y_v_job = enc.fit_transform(y_v_job)


In [ ]:
y_v_job_pred = rfc.predict(X_v_job)
accuracy_score(y_pred=y_v_job_pred, y_true=y_v_job)

0.29451540195341847

In [ ]:
hyperparam_grid={"max_features":[3,4,5,6,7],
                 'min_samples_leaf':[1,3,5,7,9,11]}

best_score=0

# takes line 3 minutes to run!!!!
for g in ParameterGrid(hyperparam_grid):
    rfc.set_params(**g)
    #or rfc.set_params(min_samples_leaf=g['min_samples_leaf'], max_features=g['max_features'])
    rfc.fit(X_tr_job,y_tr_job)
    # save if best
    y_pred = rfc.predict(X=X_v_job)
    acc = accuracy_score(y_true=y_v_job, y_pred=y_pred)
    print(acc)
    if acc > best_score:
        best_score = acc
        best_params = g

print(f"Accuracy: %0.5f" % best_score)
print("Best parameters:", best_params)

# Imputation for Device 
## Train data - `device` 

In [82]:
X_tr_dev = X_train.drop(columns=['job', 'education'])
X_test_dev = X_tr_dev[X_tr_dev['device'] == 'na'].drop(columns='device') 
#this is the data set that we need to predict and fuse with actual Training data for future imputation of "education"
X_test_dev


,age,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,X1,X2,X3,X4,marital_divorced,marital_married,marital_single,outcome_old_failure,outcome_old_na,outcome_old_other,outcome_old_success
Id,,,,,,,,,,,,,,,,,,
4359,44,17,6,24.05,1,0,-1,0,0,0,0.076980,0,1,0,0,1,0,0
5378,31,26,5,8.60,5,0,-1,0,0,1,0.068700,1,0,0,0,1,0,0
3095,49,5,6,0.45,3,0,-1,0,0,0,0.088029,0,1,0,0,1,0,0
8737,36,16,6,33.80,11,0,-1,0,0,1,0.075291,0,1,0,0,1,0,0
7745,32,12,5,13.95,3,0,-1,0,0,1,0.085014,0,0,1,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4510,32,6,6,29.80,5,0,-1,0,0,1,0.061255,0,1,0,0,1,0,0
4515,36,11,6,5.35,2,0,-1,0,0,0,0.095655,0,1,0,0,1,0,0
4030,29,6,5,12.90,2,0,-1,0,0,1,0.073194,0,1,0,0,1,0,0


In [27]:
X_tr_dev = X_tr_dev[X_tr_dev['device'] != 'na']# this is the data set that does not have NAs
y_tr_dev = X_tr_dev[['device']] 
X_tr_dev = X_tr_dev.drop(columns='device')


In [50]:
X_tr_dev

,age,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,X1,X2,X3,X4,marital_divorced,marital_married,marital_single,outcome_old_failure,outcome_old_na,outcome_old_other,outcome_old_success
Id,,,,,,,,,,,,,,,,,,
4949,59,6,8,3.90,2,0,-1,0,0,0,0.073675,0,1,0,0,1,0,0
4298,50,5,8,27.15,1,0,-1,0,0,0,0.093803,0,1,0,0,1,0,0
6904,55,21,11,2.25,5,0,-1,1,0,1,0.085450,0,1,0,0,1,0,0
4642,44,19,11,17.65,2,0,-1,0,0,0,0.086812,0,1,0,0,1,0,0
2403,53,19,11,27.55,1,0,-1,0,0,0,0.078133,1,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8241,27,4,2,53.40,1,0,-1,0,0,1,0.081256,0,0,1,0,1,0,0
278,42,12,5,0.70,1,0,-1,0,0,0,0.072803,0,1,0,0,1,0,0
3714,39,13,5,98.25,5,0,-1,1,0,1,0.072803,0,0,1,0,1,0,0


In [28]:
enc = OrdinalEncoder()
y_tr_dev = enc.fit_transform(y_tr_dev)
y_tr_dev

array([[1.],
       [1.],
       [0.],
       ...,
       [1.],
       [1.],
       [1.]])

In [36]:
rfc_dev = RandomForestClassifier(n_estimators=100, random_state=59, n_jobs=-2)

In [37]:
rfc_dev.fit(X_tr_dev, y_tr_dev)

C:\Users\ficop\AppData\Local\Temp\ipykernel_11976\3635923302.py:1: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rfc_dev.fit(X_tr_dev, y_tr_dev)


RandomForestClassifier(n_jobs=-2, random_state=59)

In [38]:
y_tr_dev_pred = rfc_dev.predict(X_tr_dev)
accuracy_score(y_true=y_tr_dev, y_pred=y_tr_dev_pred)
#np.array(y_tr_dev_pred.shape)


1.0

## Validation `device`

In [39]:
X_v_dev = X_valid.drop(columns=['job', 'education'])
X_test_dev = X_v_dev[X_v_dev['device'] == 'na'].drop(columns='device')
X_v_dev = X_v_dev[X_v_dev['device'] != 'na']
y_v_dev = X_v_dev[['device']]
X_v_dev = X_v_dev.drop(columns='device')
enc = OrdinalEncoder()
y_v_dev = enc.fit_transform(y_v_dev)

In [40]:
y_v_dev_pred = rfc_dev.predict(X_v_dev)
accuracy_score(y_pred=y_v_dev_pred, y_true=y_v_dev)


0.9148936170212766

# Update Train set

In [ ]:
X_test_dev["device"] = rfc_dev.predict(X_test_dev)
X_test_dev

In [ ]:
X_test_dev[X_test_dev["device"]==1.0] = "smartphone"
X_test_dev[X_test_dev["device"]==0.0] = "desktop"
X_test_dev

In [72]:
X_test_dev

,age,day,month,time_spent,banner_views,banner_views_old,days_elapsed_old,X1,X2,X3,X4,marital_divorced,marital_married,marital_single,outcome_old_failure,outcome_old_na,outcome_old_other,outcome_old_success,device
Id,,,,,,,,,,,,,,,,,,,
2970,38,28,5,5.00,5,0,-1,0,0,1,0.075336,0,1,0,0,1,0,0,1.0
4319,35,4,6,22.35,1,0,-1,0,0,1,0.083044,0,1,0,0,1,0,0,1.0
7004,32,23,5,5.05,2,0,-1,0,0,0,0.074174,0,0,1,0,1,0,0,1.0
2801,40,20,5,3.90,2,0,-1,0,0,1,0.072803,0,1,0,0,1,0,0,1.0
8608,40,5,6,3.65,3,0,-1,1,1,1,0.072903,0,1,0,0,1,0,0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
836,37,2,6,24.95,2,0,-1,0,0,1,0.078913,1,0,0,0,1,0,0,1.0
226,46,30,5,62.95,1,0,-1,0,0,1,0.204274,1,0,0,0,1,0,0,1.0
1860,45,1,6,0.25,1,0,-1,0,0,0,0.072803,1,0,0,0,1,0,0,1.0


In [71]:
X_test_dev.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 309 entries, 2970 to 1680
Data columns (total 19 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   age                  309 non-null    int64  
 1   day                  309 non-null    int64  
 2   month                309 non-null    int64  
 3   time_spent           309 non-null    float64
 4   banner_views         309 non-null    int64  
 5   banner_views_old     309 non-null    int64  
 6   days_elapsed_old     309 non-null    int64  
 7   X1                   309 non-null    int64  
 8   X2                   309 non-null    int64  
 9   X3                   309 non-null    int64  
 10  X4                   309 non-null    float64
 11  marital_divorced     309 non-null    uint8  
 12  marital_married      309 non-null    uint8  
 13  marital_single       309 non-null    uint8  
 14  outcome_old_failure  309 non-null    uint8  
 15  outcome_old_na       309 non-null   

In [47]:
oe = OneHotEncoder()
X = oe.fit_transform(X_train[["device"]])
pd.DataFrame(X.toarray())
#X_train["device"] = pd.DataFrame(X.toarray())
oe.categories_

[array(['desktop', 'na', 'smartphone'], dtype=object)]

In [80]:
new_data = pd.merge(X_train, X_test_dev[["device"]], how="left", on = "Id")

#new_data.iloc[278]

KeyError: "None of [Index(['device'], dtype='object')] are in the [columns]"

In [85]:
X_train.iloc[4360]

age                             51
job                        manager
education              grad_school
device                          na
day                              6
month                            6
time_spent                     4.0
banner_views                     4
banner_views_old                 0
days_elapsed_old                -1
X1                               0
X2                               0
X3                               0
X4                        0.072912
marital_divorced                 0
marital_married                  1
marital_single                   0
outcome_old_failure              0
outcome_old_na                   1
outcome_old_other                0
outcome_old_success              0
Name: 442, dtype: object